In [1]:
import pandas as pd

In [260]:
df = pd.read_csv("data/raw/pitchdata.csv", sep=',')

In [261]:
df.head()

,GameId,PitcherId,HitterId,PitcherSide,HitterSide,PrimaryEvent,PitcherTeamId,HitterTeamId,PA,AB,H,2B,3B,HR,TB,BB,SF,HBP
0,2016/09/15/clemlb-chamlb-1,448306,424825,R,L,GO,145,114,1,1,0,0,0,0,0,0,0,0
1,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,BHA,145,114,0,0,0,0,0,0,0,0,0,0
2,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,GO,145,114,1,1,0,0,0,0,0,0,0,0
3,2016/09/15/clemlb-chamlb-1,448306,608070,R,L,FO,145,114,1,1,0,0,0,0,0,0,0,0
4,2016/09/15/clemlb-chamlb-1,605182,594809,R,L,KS,114,145,1,1,0,0,0,0,0,0,0,0


In [262]:
df['AVG'] =  df['H']/df['AB']

In [263]:
df['SLG'] = (df['H'] + 2*df['2B'] + 3*df['3B'] + 4*df['HR'])/df['AB']

In [264]:
df['OBP'] = (df['H']+df['BB']+df['HBP']) / (df['AB']+df['BB']+df['SF']+df['HBP'])

In [265]:
df['OPS'] = df['OBP']+df['SLG']

In [266]:
df = df.fillna(0)

In [267]:
df.head()

,GameId,PitcherId,HitterId,PitcherSide,HitterSide,PrimaryEvent,PitcherTeamId,HitterTeamId,PA,AB,...,3B,HR,TB,BB,SF,HBP,AVG,SLG,OBP,OPS
0,2016/09/15/clemlb-chamlb-1,448306,424825,R,L,GO,145,114,1,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,BHA,145,114,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,GO,145,114,1,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,2016/09/15/clemlb-chamlb-1,448306,608070,R,L,FO,145,114,1,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
4,2016/09/15/clemlb-chamlb-1,605182,594809,R,L,KS,114,145,1,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [268]:
def fPitcher(row):
    if row['HitterSide']=='L':
        val = 'vs LHH'
    elif row['HitterSide']=='R':
        val = 'vs RHH'   
    return val

In [269]:
def fHitter(row):
    if row['PitcherSide']=='L':
        val = 'vs LHP'
    elif row['PitcherSide']=='R':
        val = 'vs RHP'   
    return val

In [270]:
def createDF(dfGiven, value1,value2,function):
    dfTemp = dfGiven.filter([value1,value2,'PA','AVG','SLG','OBP','OPS'], axis=1)
    dfTemp['Split'] = dfTemp.apply(function,axis=1)
    dfTemp['SubjectId'] = dfTemp[value1]
    dfTemp = dfTemp.filter(['SubjectId','Split','Subject','PA','AVG','SLG','OBP','OPS'],axis=1)
    return dfTemp

In [286]:
def getCleanedDF(dfNew, stringVal):
    dfTemp = checkPA(dfNew)
    dfTeam = pd.merge(dfNew,dfTemp, on="SubjectId", how="right")
    dfTeam = dfTeam.filter(["SubjectId","Split","AVG","SLG","OBP","OPS"], axis=1)
    dfTeam = dfTeam.groupby(["SubjectId","Split"]).mean()
    dfTeam['Subject'] = stringVal
    dfTeam = dfTeam.reset_index()
    dfTeam = pd.melt(dfTeam, id_vars=["SubjectId", "Split","Subject"], var_name="Stat", value_name="Value")
    return dfTeam.round(3)

In [287]:
def checkPA(dfNew):
    dfTemp = dfNew.filter(['SubjectId','PA'], axis=1)
    dfTemp = dfTemp.groupby('SubjectId').sum()
    dfTemp = dfTemp[dfTemp['PA'] >= 25]
    dfTemp['SubjectId'] = dfTemp.index
    return dfTemp

In [288]:
df1 = createDF(df,'PitcherTeamId','HitterSide',fPitcher)
dfPitcherTeamId = getCleanedDF(df1, 'PitcherTeamId')

In [289]:
df2 = createDF(df,'HitterTeamId','PitcherSide',fHitter)
dfHitterTeamId = getCleanedDF(df2,'HitterTeamId')

In [290]:
df3 = createDF(df,'PitcherId','HitterSide',fPitcher)
dfHitterId = getCleanedDF(df3,'HitterId')

In [291]:
df4 = createDF(df,'HitterId','PitcherSide',fHitter)
dfPitcherId = getCleanedDF(df4,'PitcherId')

In [292]:
dfPitcherId.tail()

,SubjectId,Split,Subject,Stat,Value
2451,643393,vs RHP,PitcherId,OPS,0.613
2452,643603,vs LHP,PitcherId,OPS,1.500
2453,643603,vs RHP,PitcherId,OPS,0.545
2454,649557,vs LHP,PitcherId,OPS,0.143
2455,649557,vs RHP,PitcherId,OPS,0.698


In [278]:
dfTempMerge = pd.concat([dfPitcherTeamId,dfHitterTeamId], ignore_index=True)

In [279]:
dfTempMerge.shape

(480, 5)

In [293]:
def allMerge(df1,df2,df3,df4):
    dfConct = pd.concat([dfPitcherTeamId,dfHitterTeamId, dfPitcherId, dfHitterId], ignore_index=True)
    return dfConct

In [294]:
dfCombine = allMerge(dfPitcherTeamId,dfHitterTeamId, dfPitcherId, dfHitterId)

In [295]:
dfFinal = dfCombine.filter(["SubjectId","Stat","Split","Subject","Value"], axis=1)

In [296]:
dfFinal.head()

,SubjectId,Stat,Split,Subject,Value
0,108,AVG,vs LHH,PitcherTeamId,0.191
1,108,AVG,vs RHH,PitcherTeamId,0.221
2,109,AVG,vs LHH,PitcherTeamId,0.190
3,109,AVG,vs RHH,PitcherTeamId,0.191
4,110,AVG,vs LHH,PitcherTeamId,0.179


In [297]:
dfFinal = dfFinal.sort_values(["SubjectId","Stat","Split","Subject"])

In [298]:
dfFinal.head()

,SubjectId,Stat,Split,Subject,Value
0,108,AVG,vs LHH,PitcherTeamId,0.191
240,108,AVG,vs LHP,HitterTeamId,0.197
1,108,AVG,vs RHH,PitcherTeamId,0.221
241,108,AVG,vs RHP,HitterTeamId,0.199
120,108,OBP,vs LHH,PitcherTeamId,0.261


In [ ]:
dfFinal.to_csv("data/processed/output.csv",sep=',', index=False)